# QC-Py-01 : Configuration et Premier Backtest QuantConnect

**Durée estimée** : 45 minutes

**Objectifs d'apprentissage** :
1. Créer un compte QuantConnect gratuit (free tier)
2. Découvrir l'interface QuantConnect Lab dans le cloud
3. Exécuter votre premier backtest cloud avec une stratégie Moving Average
4. Comprendre l'architecture LEAN Engine
5. [OPTIONNEL] Installer LEAN CLI pour développement local

**Prérequis** :
- Connaissance de base de Python
- Compréhension élémentaire des marchés financiers
- Navigateur web moderne

---
## 1. Introduction : Qu'est-ce que QuantConnect ?

### Une plateforme cloud-first pour le trading algorithmique

**QuantConnect** est une plateforme de trading algorithmique qui combine :
- Un moteur de backtesting open-source (**LEAN Engine**)
- Une infrastructure cloud pour exécuter vos stratégies
- Des données de marché historiques gratuites (Equity, Crypto, Forex)
- Un IDE en ligne (QuantConnect Lab) avec notebooks et éditeur de code
- La possibilité de déployer en paper trading et live trading

### Architecture cloud-first vs local

| Approche | Avantages | Inconvénients |
|----------|-----------|---------------|
| **Cloud** (recommandé) | - Setup immédiat<br>- Données gratuites<br>- Backtests rapides<br>- Pas de configuration | - Dépend d'internet<br>- Free tier limité (8h/mois) |
| **Local** (LEAN CLI) | - Contrôle total<br>- GPU pour ML/DL<br>- Données custom<br>- Debugging avancé | - Configuration Docker<br>- Téléchargement données<br>- Plus complexe |

**Dans ce notebook** : Nous privilégions l'approche **cloud** pour un démarrage rapide. L'installation locale est présentée en section 5 (optionnelle).

### Objectifs de ce notebook

À la fin de ce notebook, vous aurez :
- Un compte QuantConnect fonctionnel
- Exécuté votre premier backtest dans le cloud
- Compris le cycle de vie d'un algorithme QuantConnect
- Une vision claire de l'architecture LEAN Engine

---
## 2. Création de votre compte QuantConnect (10 min)

### Étape 2.1 : S'inscrire gratuitement

1. **Visitez** [https://www.quantconnect.com/signup](https://www.quantconnect.com/signup)
2. **Créez un compte** avec votre email
3. **Confirmez votre email** (vérifiez votre boîte de réception)
4. **Connectez-vous** au QuantConnect Lab

### Free tier vs Paid tiers

| Fonctionnalité | Free Tier | Team ($20/mois) | Organization ($50/mois) |
|----------------|-----------|-----------------|-------------------------|
| **Backtesting** | 8h calcul/mois | 80h calcul/mois | 200h calcul/mois |
| **Paper trading** | Illimité | Illimité | Illimité |
| **Données** | US Equity, Crypto, Forex (2010+) | + Alternative data | + Options, Futures |
| **Live trading** | Non | Oui | Oui |
| **GPU** | Non | Oui | Oui |
| **Collaborateurs** | 1 | 3 | 10 |

**Note** : Le free tier est largement suffisant pour cette série de notebooks. Nous fournirons des workarounds pour les fonctionnalités payantes (ex: NewsAPI gratuit pour sentiment analysis).

### Tour de l'interface QuantConnect Lab

Une fois connecté, vous verrez :

- **Left Panel** : Explorateur de fichiers, projets, bibliothèques
- **Center Panel** : Éditeur de code (Python/C#) ou notebook Jupyter
- **Right Panel** : Console de logs, résultats de backtests
- **Top Bar** : Boutons "Backtest", "Research", "Optimize", "Deploy"

**Astuce** : QuantConnect Lab supporte deux modes :
1. **Algorithm mode** : Fichiers `.py` ou `.cs` pour stratégies
2. **Research mode** : Notebooks Jupyter pour exploration de données

### Créer votre premier projet

1. **File → New Project**
2. **Nom** : `My First Strategy`
3. **Langage** : Python
4. **Template** : `Basic Template Algorithm`

Vous verrez un fichier `main.py` avec une structure de base. Nous allons le remplacer dans la section 3.

---
## 3. Premier backtest cloud : Moving Average Crossover (15 min)

### Stratégie : Moving Average Crossover

La stratégie **Moving Average Crossover** est un classique du trading algorithmique :

**Principe** :
- Calculer deux moyennes mobiles : **rapide** (50 jours) et **lente** (200 jours)
- **Signal d'achat** : Quand la moyenne rapide croise au-dessus de la moyenne lente
- **Signal de vente** : Quand la moyenne rapide croise en-dessous de la moyenne lente

**Asset** : SPY (ETF S&P 500)

**Période** : 2020-2023 (4 ans de données)

### Code complet de l'algorithme

Copiez ce code dans votre projet QuantConnect Lab (`main.py`) :

In [ ]:
# Ce code est à copier dans QuantConnect Lab (main.py)
# Il ne peut PAS être exécuté dans ce notebook Jupyter local

from AlgorithmImports import *

class MovingAverageCrossover(QCAlgorithm):
    """
    Moving Average Crossover Strategy
    
    Buy when fast MA crosses above slow MA
    Sell when fast MA crosses below slow MA
    """
    
    def Initialize(self):
        """Initialize algorithm: set dates, cash, assets, indicators"""
        # Set backtest period
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        
        # Set initial capital
        self.SetCash(100000)
        
        # Add SPY (S&P 500 ETF) with daily resolution
        self.symbol = self.AddEquity("SPY", Resolution.Daily).Symbol
        
        # Create moving average indicators
        self.fast = self.SMA(self.symbol, 50)  # Fast MA: 50 days
        self.slow = self.SMA(self.symbol, 200) # Slow MA: 200 days
        
        # Log initialization
        self.Debug(f"Algorithm initialized: {self.StartDate} to {self.EndDate}")
    
    def OnData(self, data):
        """Event handler called on each data slice (daily here)"""
        # Wait until slow MA is ready (needs 200 data points)
        if not self.slow.IsReady:
            return
        
        # Get current indicator values
        fast_value = self.fast.Current.Value
        slow_value = self.slow.Current.Value
        
        # Trading logic
        if fast_value > slow_value:
            # Fast MA above slow MA: bullish signal
            if not self.Portfolio.Invested:
                self.SetHoldings(self.symbol, 1.0)  # Invest 100% of portfolio
                self.Debug(f"{self.Time}: BUY - Fast MA ({fast_value:.2f}) > Slow MA ({slow_value:.2f})")
        else:
            # Fast MA below slow MA: bearish signal
            if self.Portfolio.Invested:
                self.Liquidate(self.symbol)  # Sell all holdings
                self.Debug(f"{self.Time}: SELL - Fast MA ({fast_value:.2f}) < Slow MA ({slow_value:.2f})")

### Explication ligne par ligne

#### Imports
```python
from AlgorithmImports import *
```
- Importe toutes les classes QuantConnect (QCAlgorithm, indicators, etc.)

#### Classe principale
```python
class MovingAverageCrossover(QCAlgorithm):
```
- Toutes les stratégies héritent de `QCAlgorithm`
- Cette classe expose les méthodes de lifecycle (Initialize, OnData)

#### Initialize() : Configuration de l'algorithme
```python
def Initialize(self):
    self.SetStartDate(2020, 1, 1)
    self.SetEndDate(2023, 12, 31)
    self.SetCash(100000)
```
- **SetStartDate/SetEndDate** : Période de backtest
- **SetCash** : Capital initial ($100,000)

```python
    self.symbol = self.AddEquity("SPY", Resolution.Daily).Symbol
```
- **AddEquity** : Ajoute un actif (ici SPY, ETF S&P 500)
- **Resolution.Daily** : Données journalières (vs Minute, Hour)

```python
    self.fast = self.SMA(self.symbol, 50)
    self.slow = self.SMA(self.symbol, 200)
```
- **SMA** : Simple Moving Average (moyenne mobile simple)
- Périodes : 50 jours (rapide) et 200 jours (lente)
- LEAN met à jour automatiquement ces indicateurs à chaque nouvelle donnée

#### OnData() : Logique de trading
```python
def OnData(self, data):
    if not self.slow.IsReady:
        return
```
- **OnData** : Appelé à chaque nouvelle donnée (ici une fois par jour)
- **IsReady** : L'indicateur SMA(200) a besoin de 200 points de données avant d'être valide

```python
    if fast_value > slow_value:
        if not self.Portfolio.Invested:
            self.SetHoldings(self.symbol, 1.0)
```
- **fast_value > slow_value** : Signal haussier (golden cross)
- **Portfolio.Invested** : Vérifie si on détient déjà l'actif
- **SetHoldings(symbol, 1.0)** : Investir 100% du capital dans l'actif

```python
    else:
        if self.Portfolio.Invested:
            self.Liquidate(self.symbol)
```
- **fast_value < slow_value** : Signal baissier (death cross)
- **Liquidate** : Vendre toutes les positions sur l'actif

### Lancer le backtest dans QuantConnect Lab

1. **Coller le code** ci-dessus dans `main.py` de votre projet
2. **Cliquer sur "Backtest"** en haut à droite
3. **Attendre 10-30 secondes** (le backtest s'exécute sur le cloud)
4. **Analyser les résultats** dans le panneau de droite

### Analyser les résultats

Vous verrez plusieurs graphiques et métriques :

**Equity curve** :
- Courbe de votre capital au fil du temps
- Comparaison avec le benchmark (SPY buy-and-hold)

**Métriques clés** :
| Métrique | Description | Valeur typique |
|----------|-------------|----------------|
| **Sharpe Ratio** | Rendement ajusté au risque (plus élevé = mieux) | 0.5 - 2.0 |
| **Drawdown** | Perte maximale depuis un pic (plus bas = mieux) | 10-30% |
| **Total Return** | Rendement total sur la période | 20-60% |
| **Win Rate** | Pourcentage de trades gagnants | 30-60% |

**Questions à se poser** :
- Notre stratégie bat-elle le buy-and-hold (SPY) ?
- Le Sharpe ratio est-il supérieur à 1 ?
- Le drawdown maximum est-il acceptable (<30%) ?
- Combien de trades ont été exécutés ?

> **Note importante** : Cette stratégie simple est à but pédagogique. En conditions réelles, elle nécessiterait du risk management (stop-loss, position sizing), de l'optimisation de paramètres, et des tests sur plusieurs périodes.

---
## 4. Architecture LEAN Engine (10 min)

### Composants de l'écosystème LEAN

**LEAN** (Lean Engine for Algorithmic Networks) est le moteur open-source qui fait tourner QuantConnect. Voici son architecture :

```
┌─────────────────────────────────────────────────────────┐
│                  VOTRE ALGORITHME                       │
│            (Python/C#, hérite de QCAlgorithm)           │
└────────────────────┬────────────────────────────────────┘
                     │
                     ▼
┌─────────────────────────────────────────────────────────┐
│                   LEAN ENGINE                           │
│  ┌──────────┐  ┌───────────┐  ┌──────────┐            │
│  │ Data     │  │ Algorithm │  │ Portfolio│            │
│  │ Manager  │→ │ Manager   │→ │ Manager  │            │
│  └──────────┘  └───────────┘  └──────────┘            │
│       ▲              ▲               ▲                  │
└───────┼──────────────┼───────────────┼──────────────────┘
        │              │               │
   ┌────┴───┐    ┌────┴────┐    ┌─────┴─────┐
   │ Data   │    │ Events  │    │ Brokerage │
   │ Feeds  │    │ Queue   │    │ Interface │
   └────────┘    └─────────┘    └───────────┘
```

### Composants LEAN en détail

| Composant | Rôle |
|-----------|------|
| **LEAN Engine** | Orchestrateur principal, gère le flux événementiel |
| **Data Manager** | Charge et distribue les données de marché (historiques ou temps réel) |
| **Algorithm Manager** | Exécute votre algorithme (Initialize, OnData, etc.) |
| **Portfolio Manager** | Gère les positions, le cash, les ordres |
| **Data Feeds** | Sources de données (fichiers CSV, APIs, brokers) |
| **Brokerage Interface** | Connexion au broker pour live trading (IB, TD Ameritrade, etc.) |
| **API Cloud** | Endpoints REST pour déployer/monitorer des algos |

### Lifecycle d'un algorithme : QCAlgorithm

Voici les méthodes principales que vous pouvez surcharger :

```python
class MyAlgorithm(QCAlgorithm):
    
    def Initialize(self):
        """Called once at start - setup dates, cash, assets, indicators"""
        pass
    
    def OnData(self, data):
        """Called on each new data slice (bar, tick, etc.)"""
        pass
    
    def OnOrderEvent(self, orderEvent):
        """Called when an order is filled/cancelled"""
        pass
    
    def OnEndOfDay(self, symbol):
        """Called at end of each trading day"""
        pass
    
    def OnEndOfAlgorithm(self):
        """Called once at end - cleanup, final logs"""
        pass
```

### Modèle événementiel

LEAN fonctionne en mode **événementiel** (event-driven) :

1. **Chargement des données** : LEAN charge les données de marché (historiques ou temps réel)
2. **Boucle temporelle** : Pour chaque timestamp (ex: chaque jour à 16h pour Resolution.Daily)
3. **Appel OnData** : Votre algorithme reçoit les nouvelles données via `OnData(data)`
4. **Décisions de trading** : Vous décidez d'acheter/vendre (SetHoldings, Liquidate, etc.)
5. **Exécution ordres** : LEAN simule l'exécution (slippage, commissions)
6. **Mise à jour portfolio** : Les positions et le cash sont mis à jour
7. **Répéter** jusqu'à la fin de la période

**Avantages du modèle événementiel** :
- Même code pour backtest et live trading
- Gestion naturelle du temps (pas de "look-ahead bias")
- Facile d'intégrer plusieurs sources de données

### Diagramme de flux

```
START
  ↓
Initialize()
  ↓
Load historical data (2020-2023)
  ↓
FOR each day in [2020-01-01 → 2023-12-31]:
  ↓
  OnData(data)  ← Your trading logic here
  ↓
  Execute orders (if any)
  ↓
  Update portfolio (positions, cash)
  ↓
  OnEndOfDay() [optional]
  ↓
NEXT day
  ↓
OnEndOfAlgorithm()
  ↓
Generate results (charts, metrics)
  ↓
END
```

---
## 5. [OPTIONNEL] Installation LEAN CLI local (5 min)

Cette section est pour les utilisateurs avancés qui souhaitent développer en local avec VSCode et Docker.

### Pourquoi développer en local ?

| Avantage | Description |
|----------|-------------|
| **GPU** | Entraîner des modèles Deep Learning en local |
| **Custom data** | Utiliser vos propres données (CSV, APIs) |
| **Debugging** | Debugger Python/C# avec votre IDE préféré |
| **Offline** | Développer sans internet |
| **Contrôle total** | Pas de limites de temps de calcul |

### Prérequis

Avant de commencer, assurez-vous d'avoir :
- **Python 3.11+** installé
- **Docker Desktop** installé et en cours d'exécution
- **Git** pour cloner le repository CoursIA

### Étape 1 : Installer LEAN CLI

Le LEAN CLI est l'outil officiel pour développer en local.

In [ ]:
# Installation LEAN CLI (à exécuter dans un terminal)
# Décommentez et exécutez si vous voulez tester

# !pip install lean

### Étape 2 : Authentification QuantConnect

Pour synchroniser avec le cloud (optionnel), connectez-vous :

In [ ]:
# Authentification (à exécuter dans un terminal)
# Vous devrez entrer votre User ID et API Token
# (Obtenables dans Account → API Access sur quantconnect.com)

# !lean login

### Étape 3 : Initialiser un workspace

Créez un dossier de travail pour vos algorithmes locaux :

In [ ]:
# Initialiser un workspace LEAN (à exécuter dans un terminal)

# !lean init

Cela crée la structure suivante :
```
lean-workspace/
├── lean.json          # Configuration LEAN
├── data/              # Données de marché locales
├── results/           # Résultats de backtests
└── projects/          # Vos algorithmes
```

### Étape 4 : Créer un projet Python

In [ ]:
# Créer un nouveau projet (à exécuter dans un terminal)

# !lean project-create --language python MyFirstStrategy

Cela crée un fichier `projects/MyFirstStrategy/main.py` avec un template de base.

### Étape 5 : Télécharger des données (optionnel)

Pour backtester en local, vous devez télécharger les données de marché :

In [ ]:
# Télécharger données US Equity (gratuit, ~5 Go)
# Avertissement : Cela peut prendre 30-60 minutes

# !lean data download --dataset us-equity --start 20200101 --end 20231231

### Étape 6 : Exécuter un backtest local

In [ ]:
# Backtest local (à exécuter dans un terminal)

# !lean backtest MyFirstStrategy

Le backtest s'exécute dans un conteneur Docker. Les résultats sont dans `results/MyFirstStrategy/`.

### Référence complète

Pour plus de détails sur l'installation locale, consultez :
- [GETTING-STARTED.md](../GETTING-STARTED.md) dans ce repository
- [Documentation LEAN CLI officielle](https://www.quantconnect.com/docs/v2/lean-cli)

**Note** : Le développement cloud est recommandé pour cette série de notebooks. L'installation locale est un bonus pour les utilisateurs avancés.

---
## 6. Conclusion

### Récapitulatif

Dans ce notebook, vous avez :

✅ **Créé un compte QuantConnect** gratuit (free tier)

✅ **Découvert l'interface QuantConnect Lab** dans le cloud

✅ **Exécuté votre premier backtest** avec une stratégie Moving Average Crossover sur SPY (2020-2023)

✅ **Compris l'architecture LEAN Engine** :
- Composants (Data Manager, Algorithm Manager, Portfolio Manager)
- Lifecycle (Initialize → OnData → OnEndOfAlgorithm)
- Modèle événementiel (event-driven)

✅ **(Optionnel) Découvert LEAN CLI** pour développement local

### Prochaines étapes

Dans le prochain notebook **QC-Py-02-Platform-Fundamentals**, vous allez :
- Approfondir le cycle de vie `QCAlgorithm` (Initialize, OnData, OnEndOfDay)
- Comprendre la gestion des Securities (actions, options, futures)
- Créer une stratégie Moving Average Crossover plus robuste avec logging et gestion d'erreurs
- Explorer les méthodes de trading (MarketOrder, LimitOrder, StopMarketOrder)

### Ressources complémentaires

**Documentation QuantConnect** :
- [Documentation officielle](https://www.quantconnect.com/docs)
- [LEAN Engine GitHub](https://github.com/QuantConnect/Lean)
- [Algorithm Examples](https://www.quantconnect.com/docs/v2/writing-algorithms/algorithm-examples)

**Livre recommandé** :
- [Hands-On AI Trading with Python, QuantConnect, and AWS](https://www.hands-on-ai-trading.com) (Jared Broad et al., 2025)

**Communauté** :
- [QuantConnect Forum](https://www.quantconnect.com/forum)
- [LEAN Discussions](https://github.com/QuantConnect/Lean/discussions)

**CoursIA** :
- [README principal QuantConnect](../README.md)
- [GETTING-STARTED.md](../GETTING-STARTED.md)

---

**Félicitations !** Vous avez franchi la première étape du trading algorithmique avec QuantConnect. Dans les prochains notebooks, vous construirez des stratégies de plus en plus sophistiquées, en intégrant Machine Learning, Deep Learning, et Reinforcement Learning.

**Prêt pour la suite ?** Rendez-vous dans [QC-Py-02-Platform-Fundamentals.ipynb](QC-Py-02-Platform-Fundamentals.ipynb) !